<a href="https://colab.research.google.com/github/aworklab/everydayPython/blob/main/ex_20220807.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘의 목표
* customer analysis
* iterator

In [27]:
import pandas as pd 
import numpy as np 
import plotnine as gg
from plotnine import ggplot, aes
from datetime import datetime, date, time, timezone, timedelta

import time
import re
import os 

from functools import reduce
from collections import Counter, ChainMap, defaultdict

# customer analysis

In [3]:
os.listdir()

['.config',
 'sales_train.csv',
 'listings.csv',
 'ecommerce-data.csv',
 'shops.csv',
 'rfm_test.csv',
 'retail_data_transactions.csv',
 'items.csv',
 '.Rhistory',
 'item_categories.csv',
 'KPMG_VI_New_raw_data_update_final.xlsx',
 'retail_data_response.csv',
 'online_retail_II.xlsx',
 'sample_submission.csv',
 'test.csv',
 'sample_data']

In [41]:
df = pd.read_csv('retail_data_transactions.csv')
df.head()

,customer_id,trans_date,tran_amount
0,CS5295,11-Feb-13,35
1,CS4768,15-Mar-15,39
2,CS2122,26-Feb-13,52
3,CS1217,16-Nov-11,99
4,CS1850,20-Nov-13,78


In [42]:
df['date'] = pd.to_datetime(df['trans_date'])
df['month'] = df['date'].to_numpy().astype('datetime64[M]')
df = df[['customer_id', 'month', 'date', 'tran_amount']].rename(columns = {'customer_id':'id', 'tran_amount':'rev'})
df = df.groupby(['id','month','date'])['rev'].sum().reset_index()
df.head()

,id,month,date,rev
0,CS1112,2011-06-01,2011-06-15,56
1,CS1112,2011-08-01,2011-08-19,96
2,CS1112,2011-10-01,2011-10-02,60
3,CS1112,2012-04-01,2012-04-08,56
4,CS1112,2012-06-01,2012-06-24,52


In [43]:
df2 = df.assign(year = df['date'].dt.year,
          month = df['date'].dt.month,
          day = df['date'].dt.day,
          date_ceiling = df['date'] + pd.offsets.MonthEnd(0),
          date_floor = df['date'] - pd.offsets.MonthBegin(1)
          )
df2.head()

,id,month,date,rev,year,day,date_ceiling,date_floor
0,CS1112,6,2011-06-15,56,2011,15,2011-06-30,2011-06-01
1,CS1112,8,2011-08-19,96,2011,19,2011-08-31,2011-08-01
2,CS1112,10,2011-10-02,60,2011,2,2011-10-31,2011-10-01
3,CS1112,4,2012-04-08,56,2012,8,2012-04-30,2012-04-01
4,CS1112,6,2012-06-24,52,2012,24,2012-06-30,2012-06-01


In [49]:

res1 = []
res2 = []

for n in range(10) :

  s1 = time.time()
  df2.groupby(['id'])['date'].apply(lambda x : (x.max()-x.min()))
  e1 = time.time()  

  s2 = time.time()
  df2.groupby(['id'], group_keys = False).agg({'date': lambda x : (x.max()-x.min())})
  e2 = time.time()

  res1.append(e1-s1)
  res2.append(e2-s2)


print(f't1 : {np.mean(res1)}\nt2 : {np.mean(res2)}')

t1 : 1.16301486492157
t2 : 1.119955587387085


In [29]:
df.head()

,id,month,date,rev
0,CS1112,2011-06-15,2011-06-15,56
1,CS1112,2011-08-19,2011-08-19,96
2,CS1112,2011-10-02,2011-10-02,60
3,CS1112,2012-04-08,2012-04-08,56
4,CS1112,2012-06-24,2012-06-24,52


In [51]:
info_m = df.groupby(['id','month'])['rev'].sum().reset_index().rename(columns = {'rev':'money'})
info_m = pd.pivot_table(info_m, index = 'id', columns = 'month', values = 'money').reset_index()
info_m = pd.melt(info_m, id_vars = 'id', var_name = 'month', value_name = 'money').sort_values(['id','month'], ascending = [1,0])
info_m = info_m.groupby(['id']).head(3)
info_m = info_m.groupby(['id'])['money'].sum().reset_index()
info_m.head()

,id,money
0,CS1112,39.0
1,CS1113,220.0
2,CS1114,79.0
3,CS1115,55.0
4,CS1116,0.0


In [52]:
info_f = df.groupby(['id','month'])['rev'].count().reset_index().rename(columns = {'rev':'freq'})
info_f = pd.pivot_table(info_f, index = 'id', columns = 'month', values = 'freq').reset_index()
info_f = pd.melt(info_f, id_vars = 'id', var_name = 'month', value_name = 'freq').sort_values(['id','month'], ascending = [1,0])
info_f = info_f.groupby(['id']).head(3)
info_f = info_f.groupby(['id'])['freq'].sum().reset_index()
info_f.head()

,id,freq
0,CS1112,1.0
1,CS1113,2.0
2,CS1114,1.0
3,CS1115,1.0
4,CS1116,0.0


In [53]:
max_date = (df['date'] + pd.offsets.MonthEnd(0)).max()
max_date

Timestamp('2015-03-31 00:00:00')

In [55]:
info_r = df.groupby(['id'])['date'].apply(lambda x : (max_date - x.max()).days).reset_index().rename(columns = {'date':'recency'})
info_r.head()

,id,recency
0,CS1112,76
1,CS1113,50
2,CS1114,47
3,CS1115,26
4,CS1116,218


In [58]:
info = pd.merge(pd.merge(info_r, info_f, 'left', 'id'), info_m, 'left', 'id')
info = info.assign(r = pd.qcut(info.recency, 5, list('54321')),
                   f = pd.qcut(info.freq.rank(method = 'first'), 5, list('12345')),
                   m = pd.qcut(info.money.rank(method = 'first'), 5, list('12345'))
                   )
info['rf'] = info.r.astype(str) + info.f.astype(str)
info.head()

,id,recency,freq,money,r,f,m,rf
0,CS1112,76,1.0,39.0,3,2,3,32
1,CS1113,50,2.0,220.0,4,4,5,44
2,CS1114,47,1.0,79.0,4,2,4,42
3,CS1115,26,1.0,55.0,5,2,3,52
4,CS1116,218,0.0,0.0,1,1,1,11


# iterator

In [59]:
mat = [list('ABC'), list('DEF'), list('GHI')]
for r, rows in enumerate(mat) :
  for c, letter in enumerate(rows) :
    print(f'({r},{c})-{letter}')

(0,0)-A
(0,1)-B
(0,2)-C
(1,0)-D
(1,1)-E
(1,2)-F
(2,0)-G
(2,1)-H
(2,2)-I


In [60]:
portfolio = [
    ('GOOG', 100, 490.1),
    ('IBM', 50, 91.1),
    ('CAT', 150, 83.44),
    ('IBM', 100, 45.23),
    ('GOOG', 75, 572.45),
    ('AA', 50, 23.15)
]
portfolio

[('GOOG', 100, 490.1),
 ('IBM', 50, 91.1),
 ('CAT', 150, 83.44),
 ('IBM', 100, 45.23),
 ('GOOG', 75, 572.45),
 ('AA', 50, 23.15)]

In [61]:
a, b, c, d = Counter(), defaultdict(int), defaultdict(int), defaultdict(list)

for com, share, price in portfolio : 
  a[com] += 1
  b[com] += 1
  c[com] += share
  d[com].append((share, price))

print(a)
print(b)
print(c)
print(d)

Counter({'GOOG': 2, 'IBM': 2, 'CAT': 1, 'AA': 1})
defaultdict(<class 'int'>, {'GOOG': 2, 'IBM': 2, 'CAT': 1, 'AA': 1})
defaultdict(<class 'int'>, {'GOOG': 175, 'IBM': 150, 'CAT': 150, 'AA': 50})
defaultdict(<class 'list'>, {'GOOG': [(100, 490.1), (75, 572.45)], 'IBM': [(50, 91.1), (100, 45.23)], 'CAT': [(150, 83.44)], 'AA': [(50, 23.15)]})


In [62]:
users = [{'mail': 'gregorythomas@gmail.com', 'name': 'Brett Holland', 'sex': 'M', 'age': 73},
 {'mail': 'hintoncynthia@hotmail.com', 'name': 'Madison Martinez', 'sex': 'F', 'age': 29},
 {'mail': 'wwagner@gmail.com', 'name': 'Michael Jenkins', 'sex': 'M', 'age': 51},
 {'mail': 'daniel79@gmail.com', 'name': 'Karen Rodriguez', 'sex': 'F', 'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]
users

[{'age': 73,
  'mail': 'gregorythomas@gmail.com',
  'name': 'Brett Holland',
  'sex': 'M'},
 {'age': 29,
  'mail': 'hintoncynthia@hotmail.com',
  'name': 'Madison Martinez',
  'sex': 'F'},
 {'age': 51,
  'mail': 'wwagner@gmail.com',
  'name': 'Michael Jenkins',
  'sex': 'M'},
 {'age': 32,
  'mail': 'daniel79@gmail.com',
  'name': 'Karen Rodriguez',
  'sex': 'F'},
 {'age': 42, 'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F'}]

In [64]:
for key in list(users[0].keys()) :
  if key == list(users[0].keys())[0] :
    res = eval(f'reduce(lambda acc, cur : acc + [cur["{key}"]], users, [])')
  else :
    res = np.c_[res, eval(f'reduce(lambda acc, cur : acc + [cur["{key}"]], users, [])')]

res

array([['gregorythomas@gmail.com', 'Brett Holland', 'M', '73'],
       ['hintoncynthia@hotmail.com', 'Madison Martinez', 'F', '29'],
       ['wwagner@gmail.com', 'Michael Jenkins', 'M', '51'],
       ['daniel79@gmail.com', 'Karen Rodriguez', 'F', '32'],
       ['ujackson@gmail.com', 'Amber Rhodes', 'F', '42']], dtype='<U25')

In [65]:
{key : eval(f'reduce(lambda acc, cur : acc + [cur["{key}"]],users, [])') for key in list(users[0].keys())}

{'age': [73, 29, 51, 32, 42],
 'mail': ['gregorythomas@gmail.com',
  'hintoncynthia@hotmail.com',
  'wwagner@gmail.com',
  'daniel79@gmail.com',
  'ujackson@gmail.com'],
 'name': ['Brett Holland',
  'Madison Martinez',
  'Michael Jenkins',
  'Karen Rodriguez',
  'Amber Rhodes'],
 'sex': ['M', 'F', 'M', 'F', 'F']}